In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.url'):
    shutil.rmtree('./file/2.url')

os.makedirs('./file/2.url')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./file/1.page.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)
input_['SKU Count'] = input_['SKU Count'].astype(int)
input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =


work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            times, list_url = 0, []
            while len(list_url) < input_.loc[a, 'SKU Count']:
                times += 1
                
                for page in range(input_.loc[a, 'Page']):
                    b = 0
                    while True:
                        b += 1

                        try:
                            request_url = f'''{input_.loc[a, 'Url']}&start={page*100}&num=100'''
                            
                            resp = requests.get(request_url,
                                                headers=get_header(),
                                                proxies=get_proxy(),
                                                timeout=(10, 10))
                            
                            if resp.status_code == 200:
                                soup = BeautifulSoup(resp.text, 'lxml')
                                html = etree.HTML(str(soup))

                                list_href = html.xpath('//h2[@class="item-headline"]/a/@href')
                                if list_href:
                                    break
                        except KeyboardInterrupt:
                            break
                        except:
                            continue
            
                    # = = = = = = = = = = = = = = =

                    list_url += [f'https://www.dormanproducts.com/{href.strip()}' for href in list_href]
                    list_url = sorted(list(set(list_url)), key=list_url.index)
            
                    # = = = = = = = = = = = = = = =

                    if work.qsize() % 10 == 0 and work.qsize() != 0:
                        clear_output()

                    print(f'''[尝试次数：{b}] - {input_.loc[a, 'No']}.{request_url} > Page {page+1}\n[times：{times}] - {len(list_url)} / {input_.loc[a, 'SKU Count']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

                    # = = = = = = = = = = = = = = =
            
                    if len(list_url) == input_.loc[a, 'SKU Count']:
                        break
                        
            # = = = = = = = = = = = = = = =

            output = pd.DataFrame({'No': [i+1 for i in range(len(list_url))],
                                   'Url': list_url})

            output.to_excel(f'''./file/2.url/crawler_{input_.loc[a, 'No']}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
            df_temp.loc[0, 'Request_Url'] = request_url
            
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/url_error.xlsx''', index=False)
    print('爬虫存在error')
    print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.url/{file}',
              f'''./file/2.url/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：2

[尝试次数：28] - 2.https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=Engine%2520Oil%2520Pan&start=0&num=100 > Page 1
[times：1] - 100 / 1086
[剩余数量：0] - [当前时间：13:41:13]

[尝试次数：18] - 1.https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=wheel%2520cylinder&start=0&num=100 > Page 1
[times：1] - 100 / 1214
[剩余数量：0] - [当前时间：13:41:14]

[尝试次数：36] - 2.https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=Engine%2520Oil%2520Pan&start=100&num=100 > Page 2
[times：1] - 200 / 1086
[剩余数量：0] - [当前时间：13:41:21]

[尝试次数：39] - 1.https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=wheel%2520cylinder&start=100&num=100 > Page 2
[times：1] - 200 / 1214
[剩余数量：0] - [当前时间：13:41:23]

[尝试次数：4] - 1.https://www.dormanproducts.com/gsearch.aspx?type=keyword&origin=keyword&q=wheel%2520cylinder&start=200&num=100 > Page 3
[times：1] - 300 / 1214
[剩余数量：0] - [当前时间：13:41:30]

[尝试次数：4] - 1.https://www.dormanproducts.com/gsearch.aspx?type=key

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3697.05it/s]

Done ~
